# Complete pipeline

# Developed environment 

- Ubuntu 20.04 LTS (Windows Subsystem for Linux)
- Anaconda 3
    - Python == 3.8.12 (anaconda)
        - biopython == 1.79
        - netMHCpan (PyNetMHCpan) == 0.1.5
    - fasttree == 2.1.10c
    - mafft == 7.487
    - mmseqs2 == AVX2
- Epitope sequences retrieved from 'iedb.org'
- Protein sequences retrieved from 'NCBI influenza database'

In [15]:
import matplotlib.pyplot as plt
import statistics
import pandas as pd
import numpy as np
import os

from Bio import AlignIO, Entrez, Phylo
from Bio.Align import substitution_matrices

from multiprocessing import Pool
from itertools import repeat

# NECESSARY FUNCTIONS

## Preprocessing

In [16]:
def read_fa(prot_file):

    '''
    Read in fasta(.fa) file, separate the id and sequence
    Order keeps the same
    
    [Input]
        - prot_file = fasta file to read in
    [Return]
        - protein_id = list() of protein id (fasta headers)
        - protein_seq = list() of protein sequences
    '''
    seq_id = []
    seq = []

    with open(prot_file, 'r') as infile:
        lines = infile.readlines()
        separate_seq = ''
        cnt = 0
        while cnt != len(lines):
            
            # Protein ids
            if lines[cnt][0] == '>':
                seq_id.append(lines[cnt].strip('\n'))
                
            # Protein sequences
            else:
                separate_seq += lines[cnt].strip('\n')
                # make a cut at the end of sequence, or end of the file

                if cnt + 1 == len(lines):
                    seq.append(separate_seq)
                    separate_seq = ''
                elif lines[cnt + 1][0] == '>':
                    seq.append(separate_seq)
                    separate_seq = ''
                    
            cnt += 1

    return seq_id, seq

def out_fa(seq_id, seq, output_file):
    # *** ASSUME SEQUENCE ID HAVE PREFIX '>' FOR FASTA FILE
    with open(output_file, 'w') as outfile:
        cnt = 0
        while cnt != len(seq):
            outfile.write(seq_id[cnt] + '\n')
            outfile.write(seq[cnt] + '\n')
            cnt += 1

In [17]:
def remove_chrX(seq_id, seq):
    filtered_id, filtered_seq = [], []
    
    for cnt, individ_seq in enumerate(seq):
        # count number of ambiguous AAs in the sequence
        X_cnt = individ_seq.count("X")

        if X_cnt <= 2:
            filtered_id.append(seq_id[cnt])
            try:
                filtered_seq.append(seq[cnt])
            except:
                print(cnt)
                break
    
    return filtered_id, filtered_seq

def collapse_identical_seq(seq_id, seq):
    '''
    Intake list() of sequences, only transfer first-seen unique sequences.

    [Input]
        - sequence: list() of sequences
    [Return]
        - unique_id: list() of unique protein ids
        - unique_seq: list() of unique protein seqs
    '''
    unique_seq = []
    unique_id = []
    cnt = 0

    lengths = [len(s) for s in seq]
    mean = statistics.mean(lengths)

    while cnt != len(seq_id):
        if seq[cnt] not in unique_seq:
            unique_id.append(seq_id[cnt])
            unique_seq.append(seq[cnt])
        cnt +=1
    return unique_id, unique_seq

def cleanse_short_seq(seq_id, seq):
    
    lengths = [len(s) for s in seq]
    standard = statistics.median(lengths) * 0.9

    cleansed_id, cleansed_seq = [], []
    cnt = 0

    while cnt != len(seq_id):
        if len(seq[cnt]) > standard:
            cleansed_id.append(seq_id[cnt])
            cleansed_seq.append(seq[cnt])
        cnt += 1

    return cleansed_id, cleansed_seq 

## os.Tools (MSA, cluster, phylotree)

In [18]:
def run_clustering(file, outname, threshold):
    '''
    Linux command to run MMseqs2 - clustering sequences!
    '''
    mmseq_run = 'mmseqs easy-cluster {} {} tmp --min-seq-id {} >/dev/null'.format(file, outname, threshold)

    # Run mmseqs!
    os.system(mmseq_run)

def run_alignment(infile, outfile):
    '''
    Linux command to run MAFFT - multiple sequence alignment!
    '''
    mafft = 'mafft --auto --thread -1 {} > {}'.format(infile, outfile)

    # Run mafft!
    os.system(mafft)

def run_phylotree(infile, outfile):
    '''
    Linux command to run FastTree - phylogenetic tree construction!
    '''

    fasttree = 'fasttree -quote {} > {}'.format(infile, outfile)

    # Run fasttree!
    os.system(fasttree)

## Entrez-fetch subtypes/lineages

In [56]:
def fetch_names(seq_id):
    from Bio import Entrez

    # Always tell NCBI who you are, so that you Avoid any error messages.
    Entrez.email = "JuHyung.Lee@ugent.be"
    handle = Entrez.einfo()
    result = handle.read()

    accession_id = [id.split()[0][1:] for id in seq_id]
    handle = Entrez.efetch(db='protein', id=accession_id, rettype='gb', retmode='text')

    # EFETCH-ing multiple accession ids result in continuous strings,
    # Where every new id starts by LOCUS information.
    # Only the first element is a blank => spliced out.
    records = [record.strip() for record in handle.read().split("LOCUS")[1:]]

    info_id = []
    unique_names = {}

    for record in records:
        
        record = [line.strip() for line in record.split("\n")]
        
        # For B virus sequences, strain information only needed
        # Some B viruses don't even contain 'strain' section
        try:
            strain = [i.split("=")[-1].strip("\"") for i in record if i.split("=")[0] == '/strain'][0]
        except:
            strain = [i.split("=")[-1].strip("\"") for i in record if i.split("=")[0] == '/organism'][0]

        # Remove spaces between city names e.g. Hong Kong -> HongKong
        # Phylogenetic tools do not allow spaces in sequence names
        strain = strain.replace(" ", "")
        
        # For A virus sequences, also retrieve subtype info
        if strain[0] == 'A':
            
            # However some accession numbers do not provide explicit 'serotype' section,
            # Instead they put all necessary info in 'strain' section.
            try:
                subtype = [i.split("=")[-1].strip("\"") for i in record if i.split("=")[0] == '/serotype'][0]
                name = "{}/{}".format(strain, subtype)
            except:
                name = strain

        # For B virus sequences. try to retrieve lineages (Yamagata/Victoria), which is not easy...
        elif strain[0] == 'B':
            try:
                search1 = [line.split("=")[0].split(":")[-1] for line in record if line.split("=")[0].split(":")[0] == 'type']
                if search1[0].strip(" \"") in ['Yamagata', 'Victoria']:
                    lineage = search1[0].strip(" \"")
                else:
                    search2 = [line.split("=")[-1].split(":")[-1] for line in record if 'type' in line.split("=")[-1]]
                    if search2[0].strip(" \"") in ['Yamagata', 'Victoria']:
                        lineage = search2[0].strip(" \"")
                name = "{}/{}".format(strain, lineage)
            except:
                name = strain
        
        else:
            name = strain
        
        # Later on, phylogenetic tree tool (Fasttree) does not accept special characters '()/,
        name = name.replace("'", "_")
        
        # Same names could be there
        # Give them different numbers
        if name not in unique_names:
            unique_names[name] = 1
        else:
            unique_names[name] += 1
            name = "{}_{}".format(name, unique_names[name])
               
        info_id.append(">" + name)

    # Return the final result
    return info_id  

## Epitopes

### conervancy

In [184]:
def match_test(seq1, seq2):
    '''
    Input:
        - Two Amino-Acid sequences of EQUAL lenghth (seq1, seq2)
    Output:
        - percentage identity between two sequences
    '''
    cnt = 0
    length = len(seq1)
    matching_score = 0 # add 1 for every matching parts

    ambiguous_residues = {'B':['N', 'D'], 'J':['L', 'I'], 'Z': ['Q', 'E']}
    
    while cnt != length:
        if seq1[cnt] == seq2[cnt]:
            matching_score += 1
        
        elif seq1[cnt] in ambiguous_residues.keys():
            if seq2[cnt] in ambiguous_residues[seq1[cnt]]:
                matching_score += 0.5
            
        cnt += 1
    
    percent_identity = matching_score / length
    
    return percent_identity
    
def conservation_analysis(epitope, sequences):
    # 1 epitope vs all protein sequences 
    conservation_results = []

    for seq in sequences:
            
        cnt = 0
        max_identity = 0
            
        while cnt + len(epitope) != len(seq):
            
            sliding_window = seq[cnt : cnt + len(epitope)]
                
            # See how the sliding window in protein sequence matches the epitope
            # Use function 'def match_test' from above
            percent_identity = match_test(sliding_window, epitope)
            
            # Refresh the best matching score for each protein as the conservancy result
            if percent_identity > max_identity:
                max_identity = percent_identity
            
            # If sequences shows 100% at some time, just move onto the next one
            # For the sake of computational expense...
            if max_identity == 1:
                break
            cnt += 1
                    
        conservation_results.append(max_identity)
    return conservation_results

In [189]:
def epitope_conservancy_result_avs(all_results, seq, seqid):
    unique_subtypes = set()

    for id in seqid:
        subtype = re.search("H[1-9]*[0-9]*N[1-9]*[0-9]*", id.split('/')[-1])

        if subtype:
            unique_subtypes.add(subtype.group(0))

    unique_subtypes = list(unique_subtypes)

    fraction_80match, fraction_50match, fraction_fullmatch = [], [], []
    subtype_match = []

    for result in all_results:
        # percentage of 100% identity
        fraction_fullmatch.append(round((result.count(1.0) / (len(seq))) * 100 , 2))
        # Fraction of >80% identity = 1 or 2 AA difference
        fraction_80match.append(round(len([i for i in result if i >= 0.8]) / len(seq) * 100, 2))
        # Fraction of <50% identity = 1 or 2 AA difference
        fraction_50match.append(round(len([i for i in result if i <= 0.5]) / len(seq) * 100, 2))
   
        # IAV - Number of conserved subtypes
        conserved_ids = [seqid[i] for i,j in enumerate(result) if j == 1]
        conserved_subtypes = set()
        for ids in conserved_ids:
            subtype = re.search("H[1-9]*[0-9]*N[1-9]*[0-9]*", ids.split('/')[-1])
            if subtype:
                conserved_subtypes.add(subtype.group(0))
        conserved_subtypes = list(conserved_subtypes)
        subtype_match.append("{}/{}".format(len(conserved_subtypes), len(unique_subtypes)))  

    data = {'Epitope sequence':epitopes,
            'Epitope length': [len(i) for i in epitopes],
            'Fraction of 100% matching sequences (%)': fraction_fullmatch,
            'Fraction of >80% matching sequences (%)': fraction_80match,
            'Fraction of <50% matching sequences (%)': fraction_50match,
            '# Subtypes 100% conserved': subtype_match
            }

    output = pd.DataFrame(data = data)
    output.sort_values('Fraction of 100% matching sequences (%)', ascending= False, inplace=True)

    return output

In [183]:
def epitope_conservancy_result_h(all_results, seq, seqid):

    IAV_index = []
    IBV_index = []
    unique_subtypes = set()

    for id in seqid:
        if id[1] == 'A':
            IAV_index.append(seqid.index(id))
            subtype = re.search("H[1-9]*[0-9]*N[1-9]*[0-9]*", id.split('/')[-1])

            if subtype:
                unique_subtypes.add(subtype.group(0))
            
        elif id[1] == 'B':
            IBV_index.append(seqid.index(id))
        elif id[1] == "I":
            id = id.split("Influenza")

            if id[0] == "A":
                IAV_index.append(seqid.index(id))
            elif id[0] == "B":
                IBV_index.append(seqid.index(id))

    unique_subtypes = list(unique_subtypes)
    
    
    fraction_80match, fraction_50match, fraction_fullmatch = [], [], []
    fraction_IAVmatch, fraction_IBVmatch = [], []
    iav_subtype_match = []

    for result in all_results:
        # percentage of 100% identity
        fraction_fullmatch.append(round((result.count(1.0) / (len(seq))) * 100 , 2))
        # Fraction of >80% identity = 1 or 2 AA difference
        fraction_80match.append(round(len([i for i in result if i >= 0.8]) / len(seq) * 100, 2))
        # Fraction of <50% identity = 1 or 2 AA difference
        fraction_50match.append(round(len([i for i in result if i <= 0.5]) / len(seq) * 100, 2))

        # Fraction of >80% identity against IAV
        fraction_IAVmatch.append(round(len([i for j,i in enumerate(result) if i == 1 and j in IAV_index]) / len(IAV_index) * 100, 2))
        
        # Number of conserved subtypes
        conserved_ids = [seqid[i] for i,j in enumerate(result) if j == 1]
        conserved_subtypes = set()
        for ids in conserved_ids:
            subtype = re.search("H[1-9]*[0-9]*N[1-9]*[0-9]*", ids.split('/')[-1])
            if subtype:
                conserved_subtypes.add(subtype.group(0))
        conserved_subtypes = list(conserved_subtypes)
        iav_subtype_match.append("{}/{}".format(len(conserved_subtypes), len(unique_subtypes)))

        
        # Fraction of >80% identity against IBV
        fraction_IBVmatch.append(round(len([i for j,i in enumerate(result) if i == 1 and j in IBV_index]) / len(IBV_index) * 100, 2))

    data = {'Epitope sequence':epitopes,
            'Epitope length': [len(i) for i in epitopes],
            'Fraction of 100% matching sequences (%)': fraction_fullmatch,
            'Fraction of >80% matching sequences (%)': fraction_80match,
            'Fraction of <50% matching sequences (%)': fraction_50match,
            'Fraction of 100% matching in IAV seq (%)': fraction_IAVmatch,
            '# Subtypes 100% conserved': iav_subtype_match,
            'Fraction of 100% matching in IBV seq (%)': fraction_IBVmatch,
            }

    output = pd.DataFrame(data = data)
    output.sort_values('Fraction of 100% matching sequences (%)', ascending= False, inplace=True)

    return output

In [202]:
def epitope_conservancy_result_h2(all_results, seq, seqid):

    IAV_index = []
    IBV_index = []
    unique_subtypes = set()

    for id in seqid:
        if id[1] == 'A':
            IAV_index.append(seqid.index(id))
            subtype = re.search("H[1-9]*[0-9]*N[1-9]*[0-9]*", id.split('/')[-1])

            if subtype:
                unique_subtypes.add(subtype.group(0))
            
        elif id[1] == 'B':
            IBV_index.append(seqid.index(id))
        elif id[1] == "I":
            id = id.split("Influenza")

            if id[0] == "A":
                IAV_index.append(seqid.index(id))
            elif id[0] == "B":
                IBV_index.append(seqid.index(id))

    unique_subtypes = list(unique_subtypes)
    
    
    fraction_80match, fraction_50match, fraction_fullmatch = [], [], []
    fraction_IAVmatch, fraction_IBVmatch = [], []
    iav_subtype_match, iav_subtype_partial = [], []

    for result in all_results:
        # percentage of 100% identity
        fraction_fullmatch.append(round((result.count(1.0) / (len(seq))) * 100 , 2))
        # Fraction of >80% identity = 1 or 2 AA difference
        fraction_80match.append(round(len([i for i in result if i >= 0.8]) / len(seq) * 100, 2))
        # Fraction of <50% identity = 1 or 2 AA difference
        fraction_50match.append(round(len([i for i in result if i <= 0.5]) / len(seq) * 100, 2))

        # Fraction of >80% identity against IAV
        fraction_IAVmatch.append(round(len([i for j,i in enumerate(result) if i == 1 and j in IAV_index]) / len(IAV_index) * 100, 2))
        
        # Number of conserved subtypes
        conserved_ids = [seqid[i] for i,j in enumerate(result) if j == 1]
        conserved_subtypes = set()
        for ids in conserved_ids:
            subtype = re.search("H[1-9]*[0-9]*N[1-9]*[0-9]*", ids.split('/')[-1])
            if subtype:
                conserved_subtypes.add(subtype.group(0))
        conserved_subtypes = list(conserved_subtypes)
        iav_subtype_match.append("{}/{}".format(len(conserved_subtypes), len(unique_subtypes)))

        semi_conserved = [seqid[i] for i,j in enumerate(result) if j >= 0.8]
        semi_subtypes = set()
        for ids in semi_conserved:
            subtype = re.search("H[1-9]*[0-9]*N[1-9]*[0-9]*", ids.split('/')[-1])
            if subtype:
                semi_subtypes.add(subtype.group(0))

        semi_subtypes = list(semi_subtypes)
        iav_subtype_partial.append("{}/{}".format(len(semi_subtypes), len(unique_subtypes)))
        

        
        # Fraction of >80% identity against IBV
        fraction_IBVmatch.append(round(len([i for j,i in enumerate(result) if i == 1 and j in IBV_index]) / len(IBV_index) * 100, 2))

    data = {'Epitope sequence':epitopes,
            'Epitope length': [len(i) for i in epitopes],
            'Fraction of 100% matching sequences (%)': fraction_fullmatch,
            'Fraction of >80% matching sequences (%)': fraction_80match,
            'Fraction of <50% matching sequences (%)': fraction_50match,
            'Fraction of 100% matching in IAV seq (%)': fraction_IAVmatch,
            '# Subtypes 100% conserved': iav_subtype_match,
            '# subs 80 conserved': iav_subtype_partial,
            'Fraction of 100% matching in IBV seq (%)': fraction_IBVmatch,
            }

    output = pd.DataFrame(data = data)
    output.sort_values('Fraction of 100% matching sequences (%)', ascending= False, inplace=True)

    return output

### candidates

In [27]:
def most_frequent(seq):
    counter = {}
    unique_char = list(set(seq))
    seq = list(seq)
    
    for char in unique_char:
        counter[seq.count(char)] = char
    
    most_freq_value = sorted(counter.keys())[-1]
    most_freq_residue = counter[most_freq_value]

    return most_freq_residue

def consensus_seq(matrix):
    '''
    Intake a matrix of sequences
    Output the consensus sequence.
    '''

    columns = matrix.shape[1]
    consensus = ''

    for i in range(columns):
        col = columns[:,i]
        consensus += most_frequent(col)

    return consensus

def blosum_scoring(consensus, seq):
    # Substitution matrix for scoring sequences
    blosum62 = substitution_matrices.read('blosum62.txt')  
    score = 0

    for AA in seq:
        trans = tuple([consensus, AA])
        
        try:
            score += blosum62[trans]
        except:
            score += blosum62[trans[::-1]]
    return score

def find_experimental_epitopes(k, alignment, threshold=0.5):
    '''
    Scan through aligned sequences
    to look for new 'possible' epitopes

    Input
        - k = length of k-mer to look for
        - alignment = set of aligned sequences

    Returns
        - max_score = highest score obtained
        - max_index = index of that highest score in the alignment
        - max_seq = k-mer sequence with maximum score.
    '''
    # Record sum of blosum62 score for every 'valid' window (no indels)
    score_per_window = []
    # Record the maximum sum
    std_score_per_window = []

    # Alongside, also record the index of starting point of the window.
    # For easier tracking of best-scoring regions.
    seq_per_window = [] 

    # Record the positions
    positions = []
    
    cnt = 0
        
    while cnt + k < alignment.shape[-1]:
            
        # Extract the window
        window = alignment[:, cnt : cnt + k]
        window_score = 0
        standard_score = 0
        consensus_seq = ''
        
        # We only want amino acid residues
        # slid pass the window to next if it contains any indel
        if "-" not in window:

            # For every 'valid' window:
            # Iterate through every position (column)
            for column_cnt in range(k):
                column = window[:, column_cnt]

                #Identify the most frequent residue:
                consensus = most_frequent(column)
                consensus_seq += consensus

                #Calculate BLOSUM substitution score per column
                window_score += blosum_scoring(consensus, column)

                # Exploration: what if I add a conditional?
                standard_seq = [consensus] * len(column)
                standard_score += blosum_scoring(consensus, standard_seq)

        # By default, threshold = 0.8
        if window_score > 0 and window_score >= standard_score * threshold:
            score_per_window.append(window_score)
            std_score_per_window.append(standard_score)
            seq_per_window.append(consensus_seq)
            
            positions.append([cnt+1, cnt+k+1])
        cnt += 1

    return std_score_per_window, score_per_window, seq_per_window, positions

In [28]:
def exp_epitopes_dataframe(standards, scores, sequences):
    
    # Process more data out of them
    score_fraction = []

    cnt = 0

    while cnt != len(scores):
        frac = round((scores[cnt] / standards[cnt]) * 100, 2)
        score_fraction.append(frac)

        cnt += 1 
    
    # Make pandas dataframe
    exp_data = {'candidate sequence': sequences, 
            'length': [len(seq) for seq in sequences],
            'substitution score': scores,
            'best case': standards,
            'score (%)': score_fraction}

    exp_dataframe = pd.DataFrame(data = exp_data)
    exp_dataframe.sort_values('score (%)', ascending= False, inplace=True)

    return exp_dataframe

### non-overlaps

In [29]:
def non_overlapping_exp_epitopes(exp_epitopes, epitopes):
    '''
    A somewhat strict way to look for non-overlapping elements
    between known epitopes (retrieved from IEDB) and 
    detected candidates (conserved regions in MSA).
    '''
    non_overlap = []
    sim_cnt = []

    print(len(exp_epitopes) * len(epitopes))
    
    # Iterate through candidate epitopes (discovered from MSA)
    for exp_epitope in exp_epitopes:
        similar_cnt = 0

        # Iterate through knwon epitopes (IEDB)
        for epitope in epitopes:
        
            # If candidate and known
            if epitope in exp_epitope or exp_epitope in epitope:
                similar_cnt += 1

            # Look for more detailed points
            else:
                # Unequal lengths
                if len(epitope) != len(exp_epitope):
                    main = exp_epitope
                    slider = epitope
                    
                    # Swap main/slider if epitope > exp_epitope
                    if len(epitope) > len(exp_epitope):
                        main = epitope
                        slider = exp_epitope
                    
                    cnt = 0
                    k = len(slider)

                    while cnt + k < len(main):
                        window = main[cnt: cnt+k]
                        similarity = match_test(slider, window)

                        if similarity > 0.8:
                            similar_cnt += 1
                        cnt += 1

                # Equal lengths
                else:
                    similarity = match_test(exp_epitope, epitope)
                    if similarity > 0.8:
                        similar_cnt += 1

        sim_cnt.append(similar_cnt)
        if similar_cnt == 0:
            non_overlap.append(exp_epitope)

    if non_overlap:
        return similar_cnt, non_overlap
    else:
        return None

def non_overlapping_epitopes_simple(exp_epitopes, known_epitopes):
    """
    Just check whether the candidate epitopes from MSA do not overlap with known epitopes!
    """

    non_overlaps = []

    for exp in exp_epitopes:
        if exp not in known_epitopes:
            non_overlaps.append(exp)
    
    if non_overlaps:
        return non_overlaps
    else:
        return None

#  WORKFLOWS

In [227]:
# Manual curation of which clustering threshold to use!

human_host = {"HA": 0.98, "NA": 0.98, "M1": 0.995, "M2": 0.98, "PA": 0.99, 
              "PB1":0.99, "PB2": 0.99, "NP":0.995, "NS1":0.98, "NS2":0.99}

avswine_host = {"HA": 0.95, "NA": 0.95, "M1": 0.99, "M2": 0.96, "PA": 0.96, 
                "PB1":0.97, "PB2": 0.97, "NP":0.98, "NS1":0.96, "NS2":0.97}

avswine_tree_host = {"HA": 0.87, "NA": 0.85, "M1": 0.96, "M2": 0.91, "PA": 0.95, 
                     "PB1": 0.96, "PB2": 0.96, "NP":0.96, "NS1":0.88, "NS2":0.92}

In [343]:
prot = "NS1"
h_clust, avs_clust = human_host[prot], avswine_host[prot]
h_clust_str, avs_clust_str = int(h_clust * 100), int(avs_clust * 100)

# Conatenate small portion of avian-swine sequences to human sequence
# For phylogenetic analysis...
tree_avs_clust = avswine_tree_host[prot]
tree_avs_clust_str = int(tree_avs_clust * 100)

## Preprocessing

In [344]:
print("Bringing in the fasta file!")
human_protfile = "../sequences/original_files/{}_AB_allsub.fa".format(prot, prot)
avs_protfile = "../sequences/original_files/{}_AS_allsub.fa".format(prot, prot)

hseq_id, hseq = read_fa(human_protfile)
avseq_id, avseq = read_fa(avs_protfile)

Bringing in the fasta file!


In [345]:
print("Removing sequences with excess X . . .")
hseq_id, hseq = remove_chrX(hseq_id, hseq)
avseq_id, avseq = remove_chrX(avseq_id, avseq)
print(" Removing done!")

Removing sequences with excess X . . .
 Removing done!


In [346]:
# Remove identical sequences
print("Remove identical sequences . . .")
hseq_id, hseq = collapse_identical_seq(hseq_id, hseq)
avseq_id, avseq = collapse_identical_seq(avseq_id, avseq)
print("Removing done!")

Remove identical sequences . . .
Removing done!


In [347]:
# Remove too-short sequences
print("Remove excessively short sequences . . .")
hseq_id, hseq = cleanse_short_seq(hseq_id, hseq)
avseq_id, avseq = cleanse_short_seq(avseq_id, avseq)
print("Done!")

Remove excessively short sequences . . .
Done!


In [348]:
# Intermediate output - preprocessed file
human_collapsed = "../sequences/processed/{}_AB_clust{}_collapsed.fa".format(prot, h_clust_str)
avswine_collapsed = "../sequences/processed/{}_AS_clust{}_collapsed.fa".format(prot, avs_clust_str)
out_fa(hseq_id, hseq, human_collapsed)
out_fa(avseq_id, avseq, avswine_collapsed)

## Clustering

In [349]:
# Perform clustering
human_cluster_output_file = "../sequences/clustered/{}_AB_cluster{}".format(prot, h_clust_str)
avswine_cluster_output_file = "../sequences/clustered/{}_AS_cluster{}".format(prot, avs_clust_str)
avs_tree_cluster_output_file = "../sequences/clustered/{}_AS_cluster{}_extra".format(prot, tree_avs_clust_str)

In [350]:
print("Clusterng human-host sequences . . .")
run_clustering(file=human_collapsed, outname=human_cluster_output_file, threshold=h_clust)

print("Clusterng avian-and-swine-host sequences . . .")
run_clustering(file=avswine_collapsed, outname=avswine_cluster_output_file, threshold=avs_clust)

print("Clusterng extra small portion of avian-and-swine-host sequences . . .")
run_clustering(file=avswine_collapsed, outname=avs_tree_cluster_output_file, threshold=tree_avs_clust)

# Other files from mmseqs output not needed... remove them
os.system("rm ../sequences/clustered/{}_A*_cluster*_all_seqs*.fasta".format(prot))
os.system("rm ../sequences/clustered/{}_A*_cluster*_cluster*.tsv".format(prot))

Clusterng human-host sequences . . .
Clusterng avian-and-swine-host sequences . . .
Clusterng extra small portion of avian-and-swine-host sequences . . .


0

In [351]:
human_cluster_repseq_file = "{}_rep_seq.fasta".format(human_cluster_output_file)
avswine_cluster_repseq_file = "{}_rep_seq.fasta".format(avswine_cluster_output_file)
avswine_cluster_extra_repseq = "{}_rep_seq.fasta".format(avs_tree_cluster_output_file)

In [352]:
# Read clustered files
hseq_id, hseq = read_fa(human_cluster_repseq_file)
avseq_id, avseq = read_fa(avswine_cluster_repseq_file)
extra_id, extra_seq = read_fa(avswine_cluster_extra_repseq)

## Fetch names

In [353]:
# RE-Formalize the fasta seq names
print("Fetching human-host sequences information from Entrez . . .")
hseq_id = fetch_names(hseq_id)

print("Fetching avian and swine sequences information from Entrez . . .")
avseq_id = fetch_names(avseq_id)    

print("Fetching extra portion of avian and swine sequences information from Entrez . . .")
extra_id = fetch_names(extra_id)

print("DONE!")

Fetching human-host sequences information from Entrez . . .
Fetching avian and swine sequences information from Entrez . . .
Fetching extra portion of avian and swine sequences information from Entrez . . .
DONE!


In [354]:
# Intermediate output - name-processed file
human_processed = "../sequences/processed/{}_AB_clust{}_processed.fa".format(prot, h_clust_str)
avswine_processed = "../sequences/processed/{}_AS_clust{}_processed.fa".format(prot, avs_clust_str)
extra_processed = "../sequences/processed/{}_AS_clust{}_processed_extra.fa".format(prot, tree_avs_clust_str)
out_fa(hseq_id, hseq, human_processed)
out_fa(avseq_id, avseq, avswine_processed)
out_fa(extra_id, extra_seq, extra_processed)

In [355]:
concat_seq = "../sequences/processed/{}_concat_procssed.fasta".format(prot)
os.system("cat {} {} > {}".format(human_processed,
                                  extra_processed,
                                  concat_seq))

0

## MSA

In [356]:
# Perform Multiple Sequence Alignment
human_msa = "../sequences/msa/msa_{}_AB_clust{}.fa".format(prot, h_clust_str)
avswine_msa =  "../sequences/msa/msa_{}_AS_clust{}.fa".format(prot, avs_clust_str)
concat_msa = "../sequences/msa/msa_{}_concat.fa".format(prot)

print("Performing mafft MSA . . .")
run_alignment(infile=human_processed, outfile=human_msa)

print("Performing avswine mafft")
run_alignment(infile=avswine_processed, outfile=avswine_msa)

print("Performing MSA on concatenated . . .")
run_alignment(infile=concat_seq, outfile=concat_msa)

print("MSA done!")

Performing mafft MSA . . .


OS = linux
The number of physical cores =  6
nthread = 6
nthreadpair = 6
nthreadtb = 6
ppenalty_ex = 0
stacksize: 8192 kb
rescale = 1
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..

There are 20 ambiguous characters.
  301 / 315 (thread    1)
done.

Constructing a UPGMA tree (efffree=0) ... 
  310 / 315
done.

Progressive alignment 1/2... 
STEP   314 / 314 (thread    1) h
done.

Making a distance matrix from msa.. 
  300 / 315 (thread    5)
done.

Constructing a UPGMA tree (efffree=1) ... 
  310 / 315
done.

Progressive alignment 2/2... 
STEP   314 / 314 (thread    1) h
done.

disttbfast (aa) Version 7.487
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
6 thread(s)

rescale = 1
  300 / 315 (thread    4)dndpre (aa) Version 7.487
alg=X, model=BLOSUM62, 1.53, +0.12, -0.00, noshift, amax=0.0
6 thread(s)

minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 6
randomseed = 0
blosum 62 / kimura 200
poffset = 0
niter = 2
sueff_global = 0.100000
nadd = 2

Performing avswine mafft


OS = linux
The number of physical cores =  6
nthread = 6
nthreadpair = 6
nthreadtb = 6
ppenalty_ex = 0
stacksize: 8192 kb
rescale = 1
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..

There are 22 ambiguous characters.
  601 / 646 (thread    5)
done.

Constructing a UPGMA tree (efffree=0) ... 
  640 / 646
done.

Progressive alignment 1/2... 
STEP   501 / 645 (thread    3)
Reallocating..done. *alloclen = 1475
STEP   601 / 645 (thread    1)
done.

Making a distance matrix from msa.. 
  600 / 646 (thread    3)
done.

Constructing a UPGMA tree (efffree=1) ... 
  640 / 646
done.

Progressive alignment 2/2... 
STEP   500 / 645 (thread    5)
Reallocating..done. *alloclen = 1475
STEP   601 / 645 (thread    2)
done.

disttbfast (aa) Version 7.487
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
6 thread(s)


Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --auto input > output'.
For

Performing MSA on concatenated . . .


OS = linux
The number of physical cores =  6
nthread = 6
nthreadpair = 6
nthreadtb = 6
ppenalty_ex = 0
stacksize: 8192 kb
rescale = 1
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..

There are 20 ambiguous characters.
  301 / 347 (thread    5)
done.

Constructing a UPGMA tree (efffree=0) ... 
  340 / 347
done.

Progressive alignment 1/2... 
STEP   346 / 346 (thread    3) h
done.

Making a distance matrix from msa.. 
  300 / 347 (thread    2)
done.

Constructing a UPGMA tree (efffree=1) ... 
  340 / 347
done.

Progressive alignment 2/2... 
STEP   346 / 346 (thread    0) h
done.

disttbfast (aa) Version 7.487
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
6 thread(s)

rescale = 1
  300 / 347 (thread    4)dndpre (aa) Version 7.487
alg=X, model=BLOSUM62, 1.53, +0.12, -0.00, noshift, amax=0.0
6 thread(s)

minimumweight = 0.000010
autosubalignment = 0.000000
nthread = 6
randomseed = 0
blosum 62 / kimura 200
poffset = 0
niter = 2
sueff_global = 0.100000
nadd = 2

MSA done!


In [357]:
# Perform Phylogenetic Tree construction
print("Phylogenetic tree construction . . .")
concat_phylo = "../sequences/trees/tree_{}_h{}_as{}.new".format(prot, h_clust_str, tree_avs_clust_str)
run_phylotree(infile=concat_msa, outfile=concat_phylo)

Phylogenetic tree construction . . .


FastTree Version 2.1.10 Double precision (No SSE3)
Alignment: ../sequences/msa/msa_NS1_concat.fa
Amino acid distances: BLOSUM45 Joins: balanced Support: SH-like 1000
Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1
TopHits: 1.00*sqrtN close=default refresh=0.80
ML Model: Jones-Taylor-Thorton, CAT approximation with 20 rate categories
Ignored unknown character J (seen 1 times)
Ignored unknown character X (seen 20 times)
      0.10 seconds: Joined    300 of    344
Initial topology in 0.11 seconds
Refining topology: 34 rounds ME-NNIs, 2 rounds ME-SPRs, 17 rounds ML-NNIs
      0.21 seconds: SPR round   1 of   2, 101 of 692 nodes
      0.33 seconds: SPR round   1 of   2, 401 of 692 nodes
      0.47 seconds: ME NNI round 12 of 34, 1 of 345 splits
      0.58 seconds: SPR round   2 of   2, 201 of 692 nodes
      0.71 seconds: SPR round   2 of   2, 501 of 692 nodes
Total branch-length 9.107 after 0.83 sec
      0.83 seconds: ML Lengths 1 of 345 splits
      0.97 seconds: ML Lengt

## Epitopes conservancy

In [364]:
# Re-load protein sequences
prot = "NS1"
h_clust, avs_clust = human_host[prot], avswine_host[prot]
h_clust_str, avs_clust_str = int(h_clust * 100), int(avs_clust * 100)

human_processed = "../sequences/processed/{}_AB_clust{}_processed.fa".format(prot, h_clust_str)
avswine_processed = "../sequences/processed/{}_AS_clust{}_processed.fa".format(prot, avs_clust_str)

hseq_id, hseq = read_fa(human_processed)
avseq_id, avseq = read_fa(avswine_processed)

In [365]:
# Now we look into known epitopes.. how they conserved?
epitope_file = "../sequences/epitopes/{}_AB_epitopes.csv".format(prot)
epitopes_csv = pd.read_csv(epitope_file, header=1)
epitopes = epitopes_csv["Description"].tolist()

# Some epitopes are in form of: SYQKEIQAKETMK + DEAM(Q3)
# The addition of unkown words should be gone.
epitopes = [i.split("+")[0].strip() for i in epitopes]

# I will only look for epitopes of length 9~11, typical for presentation of MHC-I alleles
epitopes = [i for i in epitopes if 9 <= len(i) <= 11]

In [366]:
from multiprocessing import Pool
from itertools import repeat

print("Processing conservancy in human host sequences . . .")
with Pool(processes=6) as pool:
    h_conservancy = pool.starmap(conservation_analysis, zip(epitopes, repeat(hseq)))

print("Processing conservancy in avian/swine host sequences . . .")
with Pool(processes=6) as pool:
    avs_conservancy = pool.starmap(conservation_analysis, zip(epitopes, repeat(avseq)))

Processing conservancy in human host sequences . . .
Processing conservancy in avian/swine host sequences . . .


In [367]:
print("Summarizing human-sequences epitope conservancy results . . .")
h_conservancy_table = epitope_conservancy_result_h2(h_conservancy, hseq, hseq_id)
print("Summarizing avian-swine sequences epitope conservancy results . . .")
avs_conservancy_table = epitope_conservancy_result_avs(avs_conservancy, avseq, avseq_id)

Summarizing human-sequences epitope conservancy results . . .
Summarizing avian-swine sequences epitope conservancy results . . .


In [368]:
h_conservancy_table

,Epitope sequence,Epitope length,Fraction of 100% matching sequences (%),Fraction of >80% matching sequences (%),Fraction of <50% matching sequences (%),Fraction of 100% matching in IAV seq (%),# Subtypes 100% conserved,# subs 80 conserved,Fraction of 100% matching in IBV seq (%)
15,GVLIGGLEW,9,71.75,81.59,15.24,84.62,8/10,10/10,0.00
5,DRLRRDQKS,9,53.02,83.17,15.24,61.54,9/10,10/10,0.00
7,GEISPLPSL,9,44.13,83.17,15.24,50.77,9/10,9/10,0.00
27,MSRDWFMLM,9,39.05,50.16,15.24,45.77,8/10,8/10,0.00
22,IILKANFSV,9,29.84,64.13,15.24,35.00,8/10,10/10,0.00
13,RLRRDQRSL,9,28.25,82.86,0.63,34.23,4/10,9/10,0.00
14,ETIVLLRAF,9,22.86,33.02,15.24,27.69,2/10,4/10,0.00
3,AIMEKNIML,9,20.63,37.46,16.19,25.00,3/10,4/10,0.00
16,IMLKANFSV,9,17.78,70.79,15.24,20.38,4/10,10/10,0.00
21,QELSDAPFL,9,16.19,53.33,15.87,19.62,2/10,9/10,0.00


In [370]:
avs_conservancy_table

,Epitope sequence,Epitope length,Fraction of 100% matching sequences (%),Fraction of >80% matching sequences (%),Fraction of <50% matching sequences (%),# Subtypes 100% conserved
5,DRLRRDQKS,9,87.00,99.23,0.00,42/43
7,GEISPLPSL,9,75.39,91.95,0.00,40/43
27,MSRDWFMLM,9,59.44,82.20,0.00,40/43
15,GVLIGGLEW,9,47.83,70.90,0.00,34/43
22,IILKANFSV,9,46.44,96.28,0.15,31/43
26,IASVPTSRY,9,23.84,56.97,4.33,13/43
12,VPASRYLTDM,10,17.34,80.96,0.62,26/43
20,MTIASVPTSRY,11,17.03,66.25,8.67,12/43
19,MTIASVPTSR,10,17.03,66.41,13.62,12/43
18,MPRQKIIGPL,10,10.22,26.78,10.22,3/43


In [371]:
h_conservancy_table.to_csv("../epitope_conservancy/{}_AB_clust{}_conservancy.csv".format(prot, h_clust_str, index=False))
avs_conservancy_table.to_csv("../epitope_conservancy/{}_AS_clust{}_conservancy.csv".format(prot, avs_clust_str, index=False))

## Candidate epitopes detection

In [ ]:
# Read MSA files
human_msa = "../sequences/msa/msa_{}_AB_clust{}.fa".format(prot, h_clust_str)
avswine_msa =  "../sequences/msa/msa_{}_AS_clust{}.fa".format(prot, avs_clust_str)

msa_hseq = AlignIO.read(human_msa, 'fasta')
msa_avseq = AlignIO.read(avswine_msa, 'fasta')

# Convert strings of sequences into list of amino acids. (1-Dimensional array -> 2-Dimensional array)
msa_hseq, msa_avseq = np.array(msa_hseq), np.array(msa_avseq)

In [ ]:
# Look into MSA - look for candidates of epitope sequences
k_mer = [k for k in range(9, 12)]

h_stds, h_scores, h_seqs, h_poss = [], [], [], []
avs_stds, avs_scores, avs_seqs, avs_poss = [], [], [], []

for k in k_mer:
    print("Looking for {}-mer candidates . . .".format(k))
    h_std, h_score, h_seq, h_pos = find_experimental_epitopes(k, msa_hseq, threshold=0.8)
    h_stds.extend(h_std)
    h_scores.extend(h_score)
    h_seqs.extend(h_seq)
    h_poss.extend(h_pos)
    
    avs_std, avs_score, avs_seq, avs_pos = find_experimental_epitopes(k, msa_avseq, threshold=0.8)
    avs_stds.extend(avs_std)
    avs_scores.extend(avs_score)
    avs_seqs.extend(avs_seq)
    avs_poss.extend(avs_pos)

Looking for 9-mer candidates . . .
Looking for 10-mer candidates . . .
Looking for 11-mer candidates . . .


In [ ]:
# Make dataframe of results!
print("Processing human sequences . . .")
h_exp_results = exp_epitopes_dataframe(h_stds, h_scores, h_seqs)
print("Processing avian-swine sequences . . .")
avs_exp_results = exp_epitopes_dataframe(avs_stds, avs_scores, avs_seqs)

Processing human sequences . . .
Processing avian-swine sequences . . .


In [ ]:
avs_exp_nonoverlap = non_overlapping_epitopes_simple(avs_seqs, epitopes)
h_exp_nonoverlap = non_overlapping_epitopes_simple(h_seqs, epitopes)

In [ ]:
h_exp_results_nonoverlap = h_exp_results.loc[h_exp_results['candidate sequence'].isin(h_exp_nonoverlap)]
h_exp_results_nonoverlap.loc[h_exp_results_nonoverlap['score (%)'] > 99]

,candidate sequence,length,substitution score,best case,score (%)


In [ ]:
avs_exp_results_nonoverlap = avs_exp_results.loc[avs_exp_results['candidate sequence'].isin(avs_exp_nonoverlap)]
avs_exp_results_nonoverlap

,candidate sequence,length,substitution score,best case,score (%)
38,KFEEIRWLI,9,24338.0,24598.0,98.94
116,QKFEEIRWLI,10,26797.0,27108.0,98.85
37,QKFEEIRWL,9,24807.0,25100.0,98.83
146,QALQLLLEVE,10,21830.0,22088.0,98.83
67,QALQLLLEV,9,19341.0,19578.0,98.79
...,...,...,...,...,...
176,DLHSLQSRNGK,11,23842.0,28614.0,83.32
23,SLQSRNGKW,9,20645.0,25100.0,82.25
21,LHSLQSRNG,9,18903.0,23092.0,81.86
100,LHSLQSRNGK,10,20885.0,25602.0,81.58


In [ ]:
# Export the results 
h_exp_results_nonoverlap.to_csv("../sequences/nonoverlap_candidates/{}_AB_clust{}_candidate_epitopes.csv".format(prot, h_clust_str, index=False))
avs_exp_results_nonoverlap.to_csv("../sequences/nonoverlap_candidates/{}_AS_clust{}_candidate_epitopes.csv".format(prot, avs_clust_str, index=False))

h_conservancy_table.to_csv("../sequences/epitope_conservancy/{}_AB_clust{}_conservancy.csv".format(prot, h_clust_str, index=False))
avs_conservancy_table.to_csv("../sequences/epitope_conservancy/{}_AS_clust{}_conservancy.csv".format(prot, avs_clust_str, index=False))